# Image Manipulation Notebook
> Image Manipulation

- title: Image Manipulation Notebook
- toc: true
- categories: [jupyter]
- image: images/tools.png

In [2]:
import javax.imageio.ImageIO;
import java.io.File;
import java.io.IOException;
import java.awt.image.BufferedImage;

public class ImageIOTest {    

    public static void main( String[] args ){
       BufferedImage img = null;  // buffer type 
        try {
            // Name of file and directories
            String name = "MonaLisa";
            String in = "images/";
            String out = "images/tmp/";

            // Either use URL or File for reading image using ImageIO
            File imageFile = new File(in + name + ".png");
            img = ImageIO.read(imageFile);  // set buffer of image data

            // ImageIO Image write to gif in Java
            // Documentation https://docs.oracle.com/javase/tutorial/2d/images/index.html
            ImageIO.write(img, "gif", new File(out + name + ".gif") );  // write buffer to gif

        } catch (IOException e) {
              e.printStackTrace();
        }
        System.out.println("Success");
    }
}
ImageIOTest.main(null);

javax.imageio.IIOException: Can't read input file!
	at java.desktop/javax.imageio.ImageIO.read(ImageIO.java:1308)
	at REPL.$JShell$34$ImageIOTest.main($JShell$34.java:48)
	at REPL.$JShell$35.do_it$($JShell$35.java:35)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at io.github.spencerpark.ijava.execution.IJavaExecutionControl.lambda$execute$1(IJavaExecutionControl.java:95)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)


Success


In [1]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String asciiFile; 
    private String greyScaleFile;
    private String redScaleFile;
    private String blueScaleFile;
    private String greenScaleFile;
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;

    // Constructor obtains attributes of picture
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.resizedFile = this.outDir + name + "." + ext;
        this.asciiFile = this.outDir + name + ".txt";
        this.greyScaleFile = this.outDir + "Grey" + name + "." + ext;
        this.redScaleFile = this.outDir + "Red" + name + "." + ext;
        this.blueScaleFile = this.outDir + "Blue" + name + "." + ext;
        this.greenScaleFile = this.outDir + "Green" + name + "." + ext;
        this.setStats();
    }

    
    // An image contains metadata, namely size, width, and height
    public void setStats() {
        BufferedImage img;
        try {
            Path path = Paths.get(this.inFile);
            this.bytes = Files.size(path);
            img = ImageIO.read(new File(this.inFile));
            this.width = img.getWidth();
            this.height = img.getHeight();
        } catch (IOException e) {
        }
    }

    // Console print of data
    public void printStats(String msg) {
        System.out.println(msg + ": " + this.bytes + " " + this.width + "x" + this.height + "  " + this.inFile);
    }

    // Convert scaled image into buffered image
    public static BufferedImage convertToBufferedImage(Image img) {

        // Create a buffered image with transparency
        BufferedImage bi = new BufferedImage(
                img.getWidth(null), img.getHeight(null),
                BufferedImage.TYPE_INT_ARGB);

        // magic?
        Graphics2D graphics2D = bi.createGraphics();
        graphics2D.drawImage(img, 0, 0, null);
        graphics2D.dispose();

        return bi;
    }
    
    // Scale or reduce to "scale" percentage provided
    public void resize(int scale) {
        BufferedImage img = null;
        Image resizedImg = null;  

        int width = (int) (this.width * (scale/100.0) + 0.5);
        int height = (int) (this.height * (scale/100.0) + 0.5);

        try {
            // read an image to BufferedImage for processing
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            // create a new BufferedImage for drawing
            resizedImg = img.getScaledInstance(width, height, Image.SCALE_SMOOTH);
        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(resizedImg), this.ext, new File(resizedFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.resizedFile;  // use scaled file vs original file in Class
        this.setStats();
    }
    
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void convertToAscii() {
        BufferedImage img = null;
        PrintWriter asciiPrt = null;
        FileWriter asciiWrt = null;

        try {
            File file = new File(this.asciiFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        try {
            asciiPrt = new PrintWriter(asciiWrt = new FileWriter(this.asciiFile, true));
        } catch (IOException e) {
            System.out.println("ASCII out file create error: " + e);
        }

        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }

        for (int i = 0; i < img.getHeight(); i+=3) {
            for (int j = 0; j < img.getWidth(); j+=1) {
                Color col = new Color(img.getRGB(j, i));
                double pixVal = (((col.getRed() * 0.30) + (col.getBlue() * 0.59) + (col
                        .getGreen() * 0.11)));
                try {
                    asciiPrt.print(asciiChar(pixVal));
                    asciiPrt.flush();
                    asciiWrt.flush();
                } catch (Exception ex) {
                }
            }
            try {
                asciiPrt.println("");
                asciiPrt.flush();
                asciiWrt.flush();
            } catch (Exception ex) {
            }
        }
    }

    // conversion table, there may be better out there ie https://www.billmongan.com/Ursinus-CS173-Fall2020/Labs/ASCIIArt
    public String asciiChar(double g) {
        String str = " ";
        if (g >= 240) {
            str = " ";
        } else if (g >= 210) {
            str = ".";
        } else if (g >= 190) {
            str = "*";
        } else if (g >= 170) {
            str = "+";
        } else if (g >= 120) {
            str = "^";
        } else if (g >= 110) {
            str = "&";
        } else if (g >= 80) {
            str = "8";
        } else if (g >= 60) {
            str = "#";
        } else {
            str = "@";
        }
        return str;
    }

    public void greyScale() {
        BufferedImage img = null;
        try {
            File file = new File(this.greyScaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }
        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                int grey = (int)(col.getRed()*0.3 + col.getGreen()*0.59 + col.getBlue()*0.11);
                try {
                    Color rgb = new Color(grey, grey, grey);
                    img.setRGB(j, i, rgb.getRGB());
                } catch (Exception ex) {
                }
            }
        }
        try {
            ImageIO.write(img, "png", new File(this.greyScaleFile));
        } catch (Exception ex) {
        }
    }

    public void redScale() {
        BufferedImage img = null;
        try {
            File file = new File(this.redScaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }
        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                int red = (int)(col.getRed());
                try {
                    Color rgb = new Color(red, 0, 0);
                    img.setRGB(j, i, rgb.getRGB());
                } catch (Exception ex) {
                }
            }
        }
        try {
            ImageIO.write(img, "png", new File(this.redScaleFile));
        } catch (Exception ex) {
        }
    }

    public void greenScale() {
        BufferedImage img = null;
        try {
            File file = new File(this.greenScaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }
        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                int green = (int)(col.getGreen());
                try {
                    Color rgb = new Color(0, green, 0);
                    img.setRGB(j, i, rgb.getRGB());
                } catch (Exception ex) {
                }
            }
        }
        try {
            ImageIO.write(img, "png", new File(this.greenScaleFile));
        } catch (Exception ex) {
        }
    }

    public void blueScale() {
        BufferedImage img = null;
        try {
            File file = new File(this.blueScaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }
        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                int blue = (int)(col.getBlue());
                try {
                    Color rgb = new Color(0, 0, blue);
                    img.setRGB(j, i, rgb.getRGB());
                } catch (Exception ex) {
                }
            }
        }
        try {
            ImageIO.write(img, "png", new File(this.blueScaleFile));
        } catch (Exception ex) {
        }
    }

    // tester/driver
    public static void main(String[] args) throws IOException {
        Pics ascii = new Pics("MonaLisa", "png");
        ascii.printStats("Original");
        ascii.resize(33);
        ascii.printStats("Scaled");
        ascii.convertToAscii();

        Pics grey = new Pics("MonaLisa", "png");
        grey.greyScale();

        Pics red = new Pics("MonaLisa", "png");
        red.redScale();
        
        Pics green = new Pics("MonaLisa", "png");
        green.greenScale();

        Pics blue = new Pics("MonaLisa", "png");
        blue.blueScale();
    }
}
Pics.main(null);

Original: 499298 389x413  images/MonaLisa.png
Scaled: 55625 128x136  images/tmp/MonaLisa.png


![](images/MonaLisa.PNG)

![](images/tmp/BlueMonaLisa.PNG)

![](images/tmp/GreenMonaLisa.PNG)

![](images/tmp/GreyMonaLisa.PNG)